### import

In [1]:
import pandas as pd
import numpy as np
from os.path import join
from datetime import datetime
import functions
import warnings
warnings.simplefilter("ignore")

In [2]:
# ################################
company = "솔림"
최종조회일 = "250831"
데이터종류 = "메모"
deposit_dtype = functions.memo_dtype
# 파일 하나만 추가할 경우 파일명만(확장자x), 여러개일 경우 ""
fn_excel = "메모조회새창_20250831"
# ################################
최초조회일 = "1993" if company == "솔림" else ""
path_excel = join(r"D:\3.자산\전산 dataset",company,"기간축적데이터", 데이터종류)
path_parquet = join(r"D:\3.자산\전산 dataset",company, "기간축적데이터", 데이터종류, "parquet")
fn_write = company + 데이터종류+"전체_" + 최초조회일 + "~" + 최종조회일 +".parquet"

In [ ]:
# parquet 파일 읽기
try : 
    df_ori = pd.read_parquet(functions.키워드로파일명찾기(path_parquet, 포함키워드=company+데이터종류+"전체", 여러파일허용=True, 전체경로=True)[-1])
except : # 파일이 없는 경우
    df_ori = pd.DataFrame(None)

# 엑셀 파일 읽기
if fn_excel =="":
    fnlist = functions.키워드로파일명찾기(path_excel, 포함키워드=데이터종류+"조회새창", 여러파일허용=True)
    df = pd.DataFrame(None)
    for fn in fnlist :
        print(f'{fn} 읽기 시작')
        df = pd.concat([df, pd.read_excel(fn, dtype=deposit_dtype)])
else : 
    fnlist = functions.키워드로파일명찾기(path_excel, fn_excel+".xlsx")
    df = pd.read_excel(fnlist, dtype=deposit_dtype)

# 합치기
df_concat = pd.concat([df_ori, df])

# parquet 저장 : 통합본 읽을 때 정렬 및 인덱스 전처리 안 함에 유의
functions.to_parquet(df_concat, join(path_parquet, fn_write))